# Import Required Modules

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from pysheds.grid import Grid

# import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
output_notebook()

## Instantiate a grid from a DEM raster
Some sample data is already included, but for extra data, see the [USGS hydrosheds project](https://hydrosheds.cr.usgs.gov/datadownload.php).

In [ ]:
# grid = Grid.from_raster('data/n30w100_con_grid/n30w100_con/n30w100_con', data_name='dem')

In [ ]:
grid = Grid.from_ascii('data/n49w1235_con_grid', data_name='dem')

In [ ]:
map_extents = grid.extent

In [ ]:
# set bokeh plot tools
tools = "pan,wheel_zoom,box_zoom,reset,tap"
TOOLTIPS = [
    ("(x,y)", "($x, $y)"),
]

# create a figure, setting the x and y ranges to the appropriate data bounds
p1 = figure(title="Test", plot_width=600, plot_height=int(400),
            x_range = map_extents[:2], y_range = map_extents[2:], 
            tools=tools, tooltips=TOOLTIPS)

min_x, max_x, min_y, max_y = map_extents

p1.image(image=[grid.dem],             # image data
         x=[min_x],               # lower left x coord
         y=[min_y],               # lower left y coord
         dw=[max_x-min_x],        # *data space* width of image
         dh=[max_y-min_y],        # *data space* height of image
#          palette="Spectral11",    # palette name
         palette="PuBuGn9",
)


show(p1)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)

plt.imshow(grid.dem, extent=grid.extent, cmap='cubehelix', zorder=1)
plt.colorbar(label='Elevation (m)')
plt.grid(zorder=0)
plt.title('Digital elevation map')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.tight_layout()
plt.savefig('img/conditioned_dem.png', bbox_inches='tight')

## Resolve flats in DEM

In [ ]:
grid.resolve_flats('dem', out_name='inflated_dem')

## Specify flow direction values

In [ ]:
#         N    NE    E    SE    S    SW    W    NW
dirmap = (64,  128,  1,   2,    4,   8,    16,  32)

In [ ]:
grid.flowdir(data='inflated_dem', out_name='dir', dirmap=dirmap)

In [ ]:
fig = plt.figure(figsize=(8,6))
fig.patch.set_alpha(0)

plt.imshow(grid.dir, extent=grid.extent, cmap='viridis', zorder=2)
boundaries = ([0] + sorted(list(dirmap)))
plt.colorbar(boundaries= boundaries,
             values=sorted(dirmap))
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Flow direction grid')
plt.grid(zorder=-1)
plt.tight_layout()
plt.savefig('img/flow_direction.png', bbox_inches='tight')

In [ ]:
grid.read_raster('data/n30w100_dir_grid/n30w100_dir/n30w100_dir', data_name='dir')

In [ ]:
grid.dir

In [ ]:
grid.dir.size

In [ ]:
fig = plt.figure(figsize=(8,6))
fig.patch.set_alpha(0)

plt.imshow(grid.dir, extent=grid.extent, cmap='viridis', zorder=2)
boundaries = ([0] + sorted(list(dirmap)))
plt.colorbar(boundaries= boundaries,
             values=sorted(dirmap))
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Flow direction grid')
plt.grid(zorder=-1)
plt.tight_layout()

## Delineate the Catchment

In [ ]:
# Specify pour point
x, y = -97.294167, 32.73750
# x, y = -97.3542, 32.6402

# Delineate the catchment
grid.catchment(data='dir', x=x, y=y, dirmap=dirmap, out_name='catch',
               recursionlimit=15000, xytype='label', nodata_out=0)

In [ ]:
# Clip the bounding box to the catchment
grid.clip_to('catch')

In [ ]:
# Get a view of the catchment
catch = grid.view('catch', nodata=np.nan)

In [ ]:
print(catch.shape)

In [ ]:
print(grid.extent)

In [ ]:
# Plot the catchment
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)

plt.grid('on', zorder=0)
im = ax.imshow(catch, extent=grid.extent, zorder=1, cmap='viridis')
plt.colorbar(im, ax=ax, boundaries=boundaries, values=sorted(dirmap), label='Flow Direction')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Delineated Catchment')
plt.savefig('img/catchment.png', bbox_inches='tight')

## Get flow accumulation

In [ ]:
grid.accumulation(data='catch', dirmap=dirmap, out_name='acc')

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)
plt.grid('on', zorder=0)
acc_img = np.where(grid.mask, grid.acc + 1, np.nan)
im = ax.imshow(acc_img, extent=grid.extent, zorder=2,
               cmap='cubehelix',
               norm=colors.LogNorm(1, grid.acc.max()))
plt.colorbar(im, ax=ax, label='Upstream Cells')
plt.title('Flow Accumulation')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.savefig('img/flow_accumulation.png', bbox_inches='tight')


## Calculate distances to upstream cells

In [ ]:
grid.flow_distance(data='catch', x=x, y=y, dirmap=dirmap, out_name='dist',
                   xytype='label', nodata_out=np.nan)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)
plt.grid('on', zorder=0)
im = ax.imshow(grid.dist, extent=grid.extent, zorder=2,
               cmap='cubehelix_r')
plt.colorbar(im, ax=ax, label='Distance to outlet (cells)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Flow Distance')
plt.savefig('img/flow_distance.png', bbox_inches='tight')